#Выбор данных

#Датасеты
- **Датасет:** [Stroke Prediction Dataset](https://www.kaggle.com/datasets/jawairia123/stroke-prediction-dataset)

 **Задача:** Прогнозирование вероятности инсульта у пациента на основе медицинских и демографических данных.

- **Датасет:** [Python Learning & Exam Performance Dataset](https://www.kaggle.com/datasets/emonsharkar/python-learning-and-exam-performance-dataset)

 **Задача:** Прогнозирование результата экзамена по Python на основе данных об обучении и активности студента.

#Метрики
- **Для классификации:**
  - **F1-score:** так как датасет может быть несбалансированным
  - **ROC-AUC:** позволяет оценить качество модели на разных порогах классификации
  - **Accuracy**
- **Для регрессии:**
  - **MAE**
  - **RMSE:** более чувствительна к большим ошибкам
  - **R²**


#Создание бейзлайна и оценка качества


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

Аналогично ЛР1

In [3]:
df_stroke = pd.read_csv('/content/healthcare-dataset-stroke-data.csv')
df_stroke = df_stroke.drop(columns=['id'])
df_exam = pd.read_csv('/content/python_learning_exam_performance.csv')
df_exam = df_exam.drop(columns=['student_id'])

df_stroke['bmi'] = df_stroke['bmi'].fillna(df_stroke['bmi'].median())

X_stroke = df_stroke.drop(columns=['stroke'])
y_stroke = df_stroke['stroke']
X_train_stroke, X_test_stroke, y_train_stroke, y_test_stroke = train_test_split(
    X_stroke, y_stroke, test_size=0.2, random_state=42, stratify=y_stroke
)

cat_cols_stroke = X_train_stroke.select_dtypes(include=['object']).columns.tolist()
num_cols_stroke = X_train_stroke.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_stroke = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols_stroke),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols_stroke)
    ]
)

X_train_stroke_processed = preprocessor_stroke.fit_transform(X_train_stroke)
X_test_stroke_processed = preprocessor_stroke.transform(X_test_stroke)
df_exam['prior_programming_experience'] = df_exam['prior_programming_experience'].fillna('No')

X_exam = df_exam.drop(columns=['final_exam_score'])
y_exam = df_exam['final_exam_score']
X_train_exam, X_test_exam, y_train_exam, y_test_exam = train_test_split(
    X_exam, y_exam, test_size=0.2, random_state=42
)

cat_cols_exam = X_train_exam.select_dtypes(include=['object']).columns.tolist()
num_cols_exam = X_train_exam.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_exam = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols_exam),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols_exam)
    ]
)

X_train_exam_processed = preprocessor_exam.fit_transform(X_train_exam)
X_test_exam_processed = preprocessor_exam.transform(X_test_exam)

Бейзлайн

In [4]:
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train_stroke_processed, y_train_stroke)
y_pred_stroke_dt = dt_clf.predict(X_test_stroke_processed)
y_pred_proba_stroke_dt = dt_clf.predict_proba(X_test_stroke_processed)[:, 1]


dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train_exam_processed, y_train_exam)
y_pred_exam_dt = dt_reg.predict(X_test_exam_processed)

# Оценка качества моделей

acc_dt = accuracy_score(y_test_stroke, y_pred_stroke_dt)
f1_dt = f1_score(y_test_stroke, y_pred_stroke_dt)
roc_auc_dt = roc_auc_score(y_test_stroke, y_pred_proba_stroke_dt)

print("=== Классификация ===")
print(f"Accuracy: {acc_dt:.4f}")
print(f"F1-Score: {f1_dt:.4f}")
print(f"ROC-AUC:  {roc_auc_dt:.4f}")
print()

mae_dt = mean_absolute_error(y_test_exam, y_pred_exam_dt)
rmse_dt = np.sqrt(mean_squared_error(y_test_exam, y_pred_exam_dt))
r2_dt = r2_score(y_test_exam, y_pred_exam_dt)

print("=== Регрессия ===")
print(f"MAE:  {mae_dt:.4f}")
print(f"RMSE: {rmse_dt:.4f}")
print(f"R²:   {r2_dt:.4f}")

=== Классификация ===
Accuracy: 0.9090
F1-Score: 0.1468
ROC-AUC:  0.5538

=== Регрессия ===
MAE:  9.4872
RMSE: 12.0742
R²:   0.5040


### **Анализ результатов**

Анализируя результаты классификации, можно отметить противоречивую картину: модель демонстрирует высокую точность (Accuracy = 0.9090), что на первый взгляд является хорошим результатом. Однако крайне низкий F1-Score (0.1468) и близкий к случайному угадыванию показатель ROC-AUC (0.5538) указывают на серьезную проблему — модель работает преимущественно с классом большинства (вероятно, здоровыми пациентами), но плохо распознает целевой класс (случаи инсульта). Это типичная ситуация несбалансированных данных, когда модель "обучается" просто предсказывать наиболее частый класс, игнорируя меньшинство.

Что касается регрессионной модели, полученные метрики (MAE = 9.49, RMSE = 12.07, R² = 0.504) свидетельствуют об умеренном качестве прогнозирования. Коэффициент детерминации R² = 0.504 показывает, что модель объясняет примерно половину дисперсии целевой переменной, что является приемлемым, но не выдающимся результатом. Разница между MAE и RMSE указывает на наличие выбросов в данных, поскольку RMSE сильнее штрафует за большие ошибки. В целом, модель требует дальнейшей оптимизации и, возможно, использования более сложных алгоритмов для улучшения точности прогнозов.

#Улучшение бейзлайна

Аналогично ЛР 1 пробуем следующие гипотезы для улучшения результатов

-   Балансировка классов
-   Подбор гиперпараметров

- Отбор признаков





In [6]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_stroke_processed, y_train_stroke)
print(f"До балансировки: {np.bincount(y_train_stroke)}")
print(f"После балансировки: {np.bincount(y_train_balanced)}")
print()

selector_clf = SelectKBest(f_classif, k=8)
X_train_selected = selector_clf.fit_transform(X_train_balanced, y_train_balanced)
X_test_selected = selector_clf.transform(X_test_stroke_processed)
print(f"Выбрано {X_train_selected.shape[1]} лучших признаков из {X_train_balanced.shape[1]}")
print()

param_grid_dt = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 5, 10, 15],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}

grid_search_dt = GridSearchCV(
    DecisionTreeClassifier(random_state=42, class_weight='balanced'),
    param_grid_dt,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)
grid_search_dt.fit(X_train_selected, y_train_balanced)

print(f"Лучшие параметры: {grid_search_dt.best_params_}")

best_dt = grid_search_dt.best_estimator_
y_pred_dt_improved = best_dt.predict(X_test_selected)
y_pred_proba_dt_improved = best_dt.predict_proba(X_test_selected)[:, 1]

acc_dt_improved = accuracy_score(y_test_stroke, y_pred_dt_improved)
f1_dt_improved = f1_score(y_test_stroke, y_pred_dt_improved)
roc_auc_dt_improved = roc_auc_score(y_test_stroke, y_pred_proba_dt_improved)

print("=== Decision Tree ===")
print(f"Accuracy:  {acc_dt_improved:.4f}")
print(f"F1-Score:  {f1_dt_improved:.4f}")
print(f"ROC-AUC:   {roc_auc_dt_improved:.4f}")



До балансировки: [3889  199]
После балансировки: [3889 3889]

Выбрано 8 лучших признаков из 16

Fitting 5 folds for each of 1050 candidates, totalling 5250 fits
Лучшие параметры: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10}
=== Decision Tree ===
Accuracy:  0.9207
F1-Score:  0.1290
ROC-AUC:   0.6608


In [7]:
selector_reg = SelectKBest(f_regression, k=10)
X_train_exam_selected = selector_reg.fit_transform(X_train_exam_processed, y_train_exam)
X_test_exam_selected = selector_reg.transform(X_test_exam_processed)
print(f"Выбрано {X_train_exam_selected.shape[1]} лучших признаков из {X_train_exam_processed.shape[1]}")
print()

param_grid_dt_reg = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 5, 10, 15],
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    'max_features': ['sqrt', 'log2', None],
    'splitter': ['best', 'random']
}

grid_search_dt_reg = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid_dt_reg,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)
grid_search_dt_reg.fit(X_train_exam_selected, y_train_exam)

print(f"Лучшие параметры: {grid_search_dt_reg.best_params_}")

best_dt_reg = grid_search_dt_reg.best_estimator_
y_pred_exam_dt_improved = best_dt_reg.predict(X_test_exam_selected)


mae_dt_improved = mean_absolute_error(y_test_exam, y_pred_exam_dt_improved)
rmse_dt_improved = np.sqrt(mean_squared_error(y_test_exam, y_pred_exam_dt_improved))
r2_dt_improved = r2_score(y_test_exam, y_pred_exam_dt_improved)

print("=== Decision Tree Regressor ===")
print(f"MAE:   {mae_dt_improved:.4f}")
print(f"RMSE:  {rmse_dt_improved:.4f}")
print(f"R²:    {r2_dt_improved:.4f}")

Выбрано 10 лучших признаков из 23

Fitting 5 folds for each of 4200 candidates, totalling 21000 fits
Лучшие параметры: {'criterion': 'poisson', 'max_depth': 15, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 2, 'splitter': 'random'}
=== Decision Tree Regressor ===
MAE:   7.5625
RMSE:  9.5383
R²:    0.6905


 **Анализ результатов**

Сравнивая результаты улучшенных моделей решающего дерева с базовыми, можно отметить существенные изменения в качестве предсказаний. В задаче классификации (прогноз инсульта) после балансировки данных с помощью SMOTE и оптимизации гиперпараметров мы получили увеличение ROC-AUC с 0.5538 до 0.6608, что указывает на улучшение способности модели различать классы. Однако показатель F1-Score снизился с 0.1468 до 0.1290, что объясняется снижением точности (Accuracy уменьшилась с 0.9090 до 0.9207) - модель стала лучше находить случаи инсульта, но при этом увеличила число ложных срабатываний.

В задаче регрессии (предсказание результатов экзаменов) наблюдается значительное улучшение всех метрик: MAE снизился с 9.4872 до 7.5625, RMSE с 12.0742 до 9.5383, а R² вырос с 0.5040 до 0.6905. Это означает, что оптимизированное дерево объясняет на 19% больше дисперсии целевой переменной и совершает меньшие ошибки в предсказаниях.

#Имплементация собвственной модели


In [13]:
class SimpleDecisionTree:
    def __init__(self, max_depth=3, task='classification'):
        self.max_depth = max_depth
        self.task = task
        self.tree = None

    def _find_best_split(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None

        n_samples, n_features = X.shape

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])

            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                if self.task == 'classification':
                    parent_impurity = 1 - max(np.bincount(y) / len(y))

                    left_impurity = 1 - max(np.bincount(y[left_mask]) / len(y[left_mask])) if len(y[left_mask]) > 0 else 0
                    right_impurity = 1 - max(np.bincount(y[right_mask]) / len(y[right_mask])) if len(y[right_mask]) > 0 else 0

                    gain = parent_impurity - (len(y[left_mask])/n_samples * left_impurity +
                                            len(y[right_mask])/n_samples * right_impurity)
                else:
                    parent_mse = np.var(y)
                    left_mse = np.var(y[left_mask]) if len(y[left_mask]) > 0 else 0
                    right_mse = np.var(y[right_mask]) if len(y[right_mask]) > 0 else 0

                    gain = parent_mse - (len(y[left_mask])/n_samples * left_mse +
                                        len(y[right_mask])/n_samples * right_mse)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold, best_gain

    def _build_tree(self, X, y, depth=0):
        if depth >= self.max_depth or len(np.unique(y)) == 1 or len(y) <= 1:
            # Лист
            if self.task == 'classification':
                values, counts = np.unique(y, return_counts=True)
                return {'leaf': True, 'value': values[np.argmax(counts)]}
            else:
                return {'leaf': True, 'value': np.mean(y)}

        feature, threshold, gain = self._find_best_split(X, y)

        if gain <= 0:
            if self.task == 'classification':
                values, counts = np.unique(y, return_counts=True)
                return {'leaf': True, 'value': values[np.argmax(counts)]}
            else:
                return {'leaf': True, 'value': np.mean(y)}

        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            'leaf': False,
            'feature': feature,
            'threshold': threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def fit(self, X, y):
        self.tree = self._build_tree(np.array(X), np.array(y))

    def _predict_one(self, x, node):
        if node['leaf']:
            return node['value']

        if x[node['feature']] <= node['threshold']:
            return self._predict_one(x, node['left'])
        else:
            return self._predict_one(x, node['right'])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in np.array(X)])

    def predict_proba(self, X):
        if self.task != 'classification':
            raise ValueError("predict_proba доступен только для классификации")
        predictions = self.predict(X)
        probs = np.zeros((len(predictions), 2))
        probs[:, 1] = predictions
        probs[:, 0] = 1 - predictions
        return probs

simple_tree_clf = SimpleDecisionTree(max_depth=5, task='classification')
simple_tree_clf.fit(X_train_stroke_processed, y_train_stroke)
y_pred_simple = simple_tree_clf.predict(X_test_stroke_processed)

acc_simple = accuracy_score(y_test_stroke, y_pred_simple)
f1_simple = f1_score(y_test_stroke, y_pred_simple)

print("\nКлассификация (Stroke Prediction):")
print(f"Accuracy: {acc_simple:.4f}")
print(f"F1-Score: {f1_simple:.4f}")
print()

simple_tree_reg = SimpleDecisionTree(max_depth=5, task='regression')
simple_tree_reg.fit(X_train_exam_processed, y_train_exam)
y_pred_simple_reg = simple_tree_reg.predict(X_test_exam_processed)

mae_simple = mean_absolute_error(y_test_exam, y_pred_simple_reg)
r2_simple = r2_score(y_test_exam, y_pred_simple_reg)

print("\nРегрессия (Exam Performance):")
print(f"MAE: {mae_simple:.4f}")
print(f"R²:  {r2_simple:.4f}")


Классификация (Stroke Prediction):
Accuracy: 0.9511
F1-Score: 0.0000


Регрессия (Exam Performance):
MAE: 7.4715
R²:  0.6971


Полученные результаты показывают, что упрощенная реализация решающего дерева демонстрирует сходную с sklearn производительность в задаче регрессии (R² 0.6971 против 0.6905 у sklearn, MAE 7.4715 против 7.5625), что свидетельствует о корректности базовой логики алгоритма, однако в задаче классификации наблюдаются критические проблемы - нулевой F1-Score при высокой Accuracy (0.9511) указывает на то, что модель полностью игнорирует минорный класс (инсульты), предсказывая только мажоритарный класс, в отличие от sklearn-реализации, которая после балансировки данных достигала F1-Score 0.1290, что говорит о необходимости доработки механизма работы с несбалансированными данными в кастомной реализации.

In [12]:
imp_my_knn_clf = SimpleDecisionTree(max_depth=5, task='classification')
imp_my_knn_clf.fit(X_train_selected, y_train_balanced)
y_pred_my_imp_clf = imp_my_knn_clf.predict(X_test_selected)

imp_my_knn_reg = SimpleDecisionTree(max_depth=15, task='regression')
imp_my_knn_reg.fit(X_train_exam_selected, y_train_exam)
y_pred_my_imp_reg = imp_my_knn_reg.predict(X_test_exam_selected)


imp_acc_my_clf = accuracy_score(y_test_stroke, y_pred_my_imp_clf)
imp_f1_my_clf = f1_score(y_test_stroke, y_pred_my_imp_clf)

print("\nКлассификация (Stroke Prediction):")
print(f"Accuracy: {imp_acc_my_clf:.4f}")
print(f"F1-Score: {imp_f1_my_clf:.4f}")

imp_mae_my_reg = mean_absolute_error(y_test_exam, y_pred_my_imp_reg)
imp_rmse_my_reg = np.sqrt(mean_squared_error(y_test_exam, y_pred_my_imp_reg))
imp_r2_my_reg = r2_score(y_test_exam, y_pred_my_imp_reg)

print("\nРегрессия (Exam Performance):")
print(f"MAE:  {imp_mae_my_reg:.4f}")
print(f"RMSE: {imp_rmse_my_reg:.4f}")
print(f"R²:   {imp_r2_my_reg:.4f}")


Классификация (Stroke Prediction):
Accuracy: 0.7407
F1-Score: 0.2319

Регрессия (Exam Performance):
MAE:  8.8307
RMSE: 11.1043
R²:   0.5805


Улчшения показали отличный эффект на задаче классификации, сильно уменьшив значение дисбаланса классов. Тем не менее, результаты регрессии только ухудшились, что может говорить о чрезмерном сокращении признаков

#Вывод
В ходе работы с решающими деревьями было продемонстрировано, что они являются эффективным инструментом как для классификации, так и для регрессии, однако требуют тщательной настройки гиперпараметров (глубины дерева, критериев разделения, минимального количества образцов в узлах) и обработки несбалансированных данных для достижения оптимальных результатов. Кастомная реализация показала работоспособность базового алгоритма, но уступила оптимизированной sklearn-версии в точности, особенно в задаче классификации, где критически важными оказались техники балансировки данных.